In [2]:
!pip install numpy pandas tensorflow

  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.5-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached h5py-3.15.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached markdown-3.10.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.5-py3-none-any.whl.metadata

In [29]:
import numpy as np # linear algebra
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import layers, models


In [30]:
with np.load(file = './mnist.npz') as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train.reshape((-1,28,28,1))
x_test = x_test.reshape((-1,28,28,1))

In [45]:
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation ='relu'),
    layers.Dense(10, activation = 'softmax')
])

In [47]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9915 - loss: 0.0256
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9937 - loss: 0.0195
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9947 - loss: 0.0167
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9954 - loss: 0.0140
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9961 - loss: 0.0109


In [48]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
prediction = model.predict(x_test[:1])
print("Prediction: ", prediction.argmax())

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9883 - loss: 0.0508
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction:  7


In [49]:
model.save('digit_recognizer.keras')

In [18]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 42.1 MB/s  0:00:01m0:00:0100:01


In [11]:
import cv2

In [50]:
model = tf.keras.models.load_model('digit_recognizer.keras')
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret: break

    img = frame[300:600, 700:1000]

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    img = cv2.resize(img, (28,28))
    img = img / 255.0
    img_final = img.reshape(1,28,28,1)

    #predict
    prediction = model.predict(img_final, verbose=0)
    digit = np.argmax(prediction)
    confidence = np.max(prediction)

    cv2.rectangle(frame, (700,300), (1000, 600), (0,255,0), 2)
    cv2.putText(frame, f"Digit: {digit} ({confidence*100:.1f}%)", (200,140), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0),2)
    cv2.imshow('Digit Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()